# Simulation of an Integrated LDPC Decoder over an AWGN channel
This code is provided as supplementary material of the lecture Channel Coding - Graph Based Codes (CC-GBC)

This code illustrates

* Generating LDPC codes according to Gallager's construction
* Use of the integrated LDPC decoder in scikit-commpy
* Monte-Carlo simulation of the error performance over a binary input AWGN channel

In [2]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix, vstack
# pip install scikit-commpy
from commpy.channelcoding.ldpc import ldpc_bp_decode
from tqdm.notebook import tqdm

Generate a parity-check matrix according to Gallager's construction

In [3]:
# generate a parity-check matrix according to Gallager's method
# do not care about 4-cycles
def generate_Gallager(dv, dc, n):
    if n % dc != 0:
        assert False, "n must be a multiple of check node degree dc"

    rows = n // dc
    # column indices
    jj = np.arange(n)
    ii = np.repeat(np.arange(rows), dc)
    Ho = coo_matrix((np.ones_like(jj), (ii, jj)), shape=(rows,n)).tocsr()
    H = Ho.copy()
    for _ in range(dv-1):
        H = vstack([H, Ho[:, np.random.permutation(n)]])
    
    return H

Carry out Monte-Carlo simulation of the error rate. Simulate 10000 frames an in each frame, we generate a new parity-check matrix. Then carry out decoding for 50 iterations and record the frame error rate

In [5]:
# parameters of regular LDPC code
dv = 3
dc = 6

# length of codeword, attention, must be an integer multiple of dc
n = 1200

# specify Es/N0 at which simulation takes place
esno_dB = -1

# number of frames to simulate
frames = 1000

# decoding iterations
iterations = 10

# compute noise standard deviation
sigma = np.sqrt(0.5 * 10**(-esno_dB/10))

# channel parameter for LLR calculation
Lc = 4*(10**(esno_dB/10))

# generate parity-check matrix of regular LDPC code
H = generate_Gallager(dv, dc, 1200)

n = H.shape[1]

# simulate all-zero codeword
bits = np.zeros(n)
x = 1 - 2*bits #BPSK 0->1 | 1->-1

# save parameters to dict
ldpc_code_params = {"parity_check_matrix": H, "n_vnodes": n}

errors = 0
for _ in tqdm(range(frames)):
    y = x + sigma*np.random.randn(n)

    # calculate LLRs
    L = Lc * y

    xh, _ = ldpc_bp_decode(llr_vec=L, ldpc_code_params=ldpc_code_params, decoder_algorithm="SPA", n_iters=iterations)

    errors = errors + np.sum(xh != bits)

BER = errors / (frames * n)
print(f"Es/N0 = {esno_dB:.2f}: BER = {BER:.4g}\n")

  0%|          | 0/1000 [00:00<?, ?it/s]

Es/N0 = -1.00: BER = 0.004659

